In [1]:
import pandas as pd



In [2]:
df = pd.read_csv('input_raw.csv')
df

,audio_clipping,audio_clipping:confidence,background_noise_audible,background_noise_audible:confidence,overall_quality_of_the_audio,quiet_speaker,quiet_speaker:confidence,speaker_id,file_download,file_name,phrase,prompt,writer_id
0,no_clipping,1.0000,light_noise,1.0000,3.33,audible_speaker,1.0,43453425,https://ml.sandbox.cf3.us/cgi-bin/index.cgi?do...,1249120_43453425_58166571.wav,When I remember her I feel down,Emotional pain,21665495
1,light_clipping,0.6803,no_noise,0.6803,3.33,audible_speaker,1.0,43719934,https://ml.sandbox.cf3.us/cgi-bin/index.cgi?do...,1249120_43719934_43347848.wav,When I carry heavy things I feel like breaking...,Hair falling out,44088126
2,no_clipping,1.0000,no_noise,0.6655,3.33,audible_speaker,1.0,43719934,https://ml.sandbox.cf3.us/cgi-bin/index.cgi?do...,1249120_43719934_53187202.wav,there is too much pain when i move my arm,Heart hurts,44292353
3,no_clipping,1.0000,light_noise,1.0000,3.33,audible_speaker,1.0,31349958,https://ml.sandbox.cf3.us/cgi-bin/index.cgi?do...,1249120_31349958_55816195.wav,My son had his lip pierced and it is swollen a...,Infected wound,43755034
4,no_clipping,1.0000,no_noise,1.0000,4.67,audible_speaker,1.0,43719934,https://ml.sandbox.cf3.us/cgi-bin/index.cgi?do...,1249120_43719934_82524191.wav,My muscles in my lower back are aching,Infected wound,21665495
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6656,no_clipping,1.0000,no_noise,0.6076,3.80,audible_speaker,1.0,14353703,https://ml.sandbox.cf3.us/cgi-bin/index.cgi?do...,1249120_14353703_45949288.wav,I feel a burning sensation in my guts about 2 ...,Stomach ache,40925305
6657,no_clipping,1.0000,no_noise,0.7829,3.80,audible_speaker,1.0,15004831,https://ml.sandbox.cf3.us/cgi-bin/index.cgi?do...,1249120_15004831_26452554.wav,I have a split on my thumb that will not heal.,Open wound,43755034
6658,no_clipping,1.0000,no_noise,0.6061,3.80,audible_speaker,1.0,15004831,https://ml.sandbox.cf3.us/cgi-bin/index.cgi?do...,1249120_15004831_64958100.wav,I feel a lot of pain in the joints.,Joint pain,43772689
6659,no_clipping,1.0000,no_noise,0.8215,3.80,audible_speaker,1.0,15830408,https://ml.sandbox.cf3.us/cgi-bin/index.cgi?do...,1249120_15830408_92962528.wav,The area around my heart doesn't feel good.,Heart hurts,44292353


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6661 entries, 0 to 6660
Data columns (total 13 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   audio_clipping                       6661 non-null   object 
 1   audio_clipping:confidence            6661 non-null   float64
 2   background_noise_audible             6661 non-null   object 
 3   background_noise_audible:confidence  6661 non-null   float64
 4   overall_quality_of_the_audio         6661 non-null   float64
 5   quiet_speaker                        6661 non-null   object 
 6   quiet_speaker:confidence             6661 non-null   float64
 7   speaker_id                           6661 non-null   int64  
 8   file_download                        6661 non-null   object 
 9   file_name                            6661 non-null   object 
 10  phrase                               6661 non-null   object 
 11  prompt                        

In [11]:
df.drop(df.columns.difference(['phrase','prompt']), axis=1, inplace=True)
df


,phrase,prompt
0,When I remember her I feel down,Emotional pain
1,When I carry heavy things I feel like breaking...,Hair falling out
2,there is too much pain when i move my arm,Heart hurts
3,My son had his lip pierced and it is swollen a...,Infected wound
4,My muscles in my lower back are aching,Infected wound
...,...,...
6656,I feel a burning sensation in my guts about 2 ...,Stomach ache
6657,I have a split on my thumb that will not heal.,Open wound
6658,I feel a lot of pain in the joints.,Joint pain
6659,The area around my heart doesn't feel good.,Heart hurts


In [14]:
uniq = df['prompt'].unique()
uniq, len(uniq)

(array(['Emotional pain', 'Hair falling out', 'Heart hurts',
        'Infected wound', 'Foot ache', 'Shoulder pain',
        'Injury from sports', 'Skin issue', 'Stomach ache', 'Knee pain',
        'Joint pain', 'Hard to breath', 'Head ache', 'Body feels weak',
        'Feeling dizzy', 'Back pain', 'Open wound', 'Internal pain',
        'Blurry vision', 'Acne', 'Muscle pain', 'Neck pain', 'Cough',
        'Ear ache', 'Feeling cold'], dtype=object),
 25)

In [15]:
df.to_csv('input_refined.csv')